In [ ]:
# Allow Colab or VSCore/Normal Jupyter environments
import os
from pathlib import Path

# The default relative path when running the notebook from a cloned repo.
LIB_PATH = Path("../lib")

if 'google.colab' in str(get_ipython()):
    print("👉 Setting up for Colab")
    # This will create a py-llm dir at the same level as this notebook
    # Refer to the lib in there using `./py-llm/lib` as opposed to the 
    # relative `../lib` when we are running straight from the py-llm/nbs 
    # directory in VScode.
    if not os.path.isdir("./py-llm"):
        print("Cloning git repo into ./py-llm")
        !git clone -b 2_lib_use_openai_and_colab https://github.com/vamsi-juvvi/py-llm.git
        LIB_PATH = Path("./py-llm/lib")
    else:
        print("./py-llm exists. Not cloning") 

In [3]:
# Append to sys.path directly
# Make sure to `str(Path)`
# - The resolve() converts relative to absolute. 
# - If you use ~ for HOME, use `Path.expand_user()`
import os, sys
import logging
from pathlib import Path

sys.path.append(str(LIB_PATH.resolve()))

from py_llm.util import jupyter_util
from py_llm.util.jupyter_util import TextAlign
from py_llm.util.jupyter_util import DisplayHTML as DH
from py_llm.util.jupyter_util import DisplayMarkdown as DM
from py_llm.util.jupyter_util import ColabEnv

# Init logging at DEBUG. Once we are done testing, this can drop 
# down to warning
jupyter_util.setup_logging(logging.INFO)

In [4]:
# Uncomment for use in Colab or when the package is missing
#!pip install -qy openai

# Final API

## module py_llm.llm.openai_util.py

```
Parent
 └──llm
   ├── __init__.py
   ├── openai_util.py
```

In [5]:
import os
import openai
import logging

openai.api_key = ColabEnv.colab_keyval_or_env("OPENAI_API_KEY")

#-------------------------------------------------------------------------------------
def get_completion(prompt, model="gpt-4o-mini", temperature=0) -> str:
    """
    Returns the one-shot completion of a simple prompt (no tools)
    as a string response.
    """
    logging.info(f"Completion Prompt : {prompt}")
    logging.debug(f"model={model}, temp={temperature}")

    chat_history = [{"role":"user", "content":prompt}]
    response = get_response(chat_history, model, temperature)    
    response_text = response.choices[0].message.content

    # Could choose to log the entire response object too.    
    logging.info(f"Response Text: {response_text}")
    return response_text

#-------------------------------------------------------------------------------------
def get_response(chat_history, model="gpt-4o-mini", temperature=0):
    """
    Returns the completion given a chat_history
    """    
    response = openai.chat.completions.create(
        model=model,
        messages=chat_history,
        tools=None,
        temperature=temperature)    
    
    return response

#-------------------------------------------------------------------------------------
def chat_history_append_response(chat_history, response):
    msg = response.choices[0].message
    chat_history.append( {
        "role" : msg.role,
        "content" : msg.content
    })

    logging.debug(f"Appended {chat_history[-1]} to chat_history")

    return chat_history

def chat_history_append_user_msg(chat_history, content):    
    chat_history.append( {
        "role" : "user",
        "content" : content
    })

    logging.debug(f"Appended {chat_history[-1]} to chat_history")

    return chat_history    

In [4]:
# Ask why the sky is blue and print the response out in a color box
completion = get_completion("Why is the sky blue")
DH.color_box(completion, "OpenAI Response")

11:33:00 INFO:Completion Prompt : Why is the sky blue
11:33:03 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
11:33:03 INFO:Response Text: The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.

As sunlight passes through the atmosphere, it collides with gas molecules and small particles. Because blue light is scattered in all directions more than other colors due to its shorter wavelength, we see a predominance of blue when we look up at the sky.

During sunrise and sunset, the sun's light has to pass through a thicker layer of the atmosphere, which scatters the shorter blue wavelengths out of our line of sight and allows the longer wavelengths, like red and orange, to dominate, resulting in the beautiful colors we see at those times.


# Experiments and playground

In [5]:
# Verify that the sys.path is as expected and contains out `py-llib/libs` path.
print("\n".join(sys.path))

/home/vamsi/mambaforge/envs/ml-pip/lib/python312.zip
/home/vamsi/mambaforge/envs/ml-pip/lib/python3.12
/home/vamsi/mambaforge/envs/ml-pip/lib/python3.12/lib-dynload

/home/vamsi/mambaforge/envs/ml-pip/lib/python3.12/site-packages
/home/vamsi/github/py-llm/lib


In [6]:
# Verify that imports work!
from py_llm.util.jupyter_util import DisplayHTML as DH
DH.color_box("Hello in colorboox")

In [7]:
# Examine the response object
chat_history = [{
    "role":"user", 
    "content":"Why is the sky blue"
    }]

response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=chat_history,
        tools=None,
        temperature=0)

11:33:16 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [8]:
# There could be multiple choices depending on the parameters
# top-k, max-p etc options. Just check the first one 
dir(response.choices[0].message)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',

In [9]:
# Ok. I see
#  role - Wanted to see if we need to manually insert the 'assistant` role when we 
#         add OpenAI response back to the chat_history or if we can simply take it 
#         from the response object itself.
#  tool_calls (which we'll get to later)
print(response.choices[0].message.role)

assistant


In [10]:
# How about the json field ? Whats that. The entire reponse in JSON format or something else ?
# Of. The following shows that it is a method
msg = response.choices[0].message
type(msg.json)

# msg.json() says deprecated and use model_dump_json() instead. This is a Pydantic model
# Prints out stringified plain json in one line. To get plain json and use our own formatting use
# msg.model_dump()
#
# The model name is so confusing here. It refers to the pydantic model (a data model), 
# not the LLM Model like "gpt-4o-mini"
import json
print(json.dumps(msg.model_dump(), indent=4))

{
    "content": "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, it collides with gas molecules and small particles. Because blue light is scattered in all directions more than other colors due to its shorter wavelength, we see a predominance of blue when we look up at the sky.\n\nDuring sunrise and sunset, the sun's light has to pass through a thicker layer of the atmosphere, which scatters the shorter blue wavelengths out of our line of sight and allows the longer wavelengths, like red and orange, to dominate, giving the sky those warm colors.",
    "refusal": null,
    "role": "assistant",
    "annotations": [],
    "audio": null,
    "function_call": null,
    "tool_calls": null
}


In [11]:
# Try out multi-turn convo out.
chat_history = [{
    "role":"user", 
    "content":"Tell me a clean joke about chefs"
    }]

r = get_response(chat_history)
chat_history_append_response(chat_history, r)
chat_history_append_user_msg(chat_history, "Summarize it")
r = get_response(chat_history)
chat_history_append_response(chat_history, r)
chat_history_append_user_msg(chat_history, "Turn it into a Haiku")
r = get_response(chat_history)
chat_history = chat_history_append_response(chat_history, r)

11:33:21 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
11:33:22 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
11:33:23 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


At this point, I decided that I can use color and HTML better and went back to [Py_mod_jupyter_utils_devel.ipynb](./Py_mod_jupyter_utils_devel.ipynb) for some more experiments. Ended up with 
 - Specifiable `fg` and `bg` which default to "black" and "pink" respectively
 - Specifiable `text alignment` with a string enum: `TextAlign`
   - Defaults to `TextAlign.DEFAULT` and `TextAlign.RIGHT` and `TextAlign.LEFT` are available
   - Enhanced later with alignment affecting the margins so that the entire box matches the text alignment. When there is a lot of text completely filling up the box, the alignment's visual effect is lost unless the box is also shortened horizontally in the right way to display the alignment.
 - Node: The imports now have to include `TextAlign` as well. Since I changed the on-disk contents of the `jupyter_util.py` module, I need to restart the kernel here to import `TextAlign`   

In [13]:
# I find it more efficient to generate the responses separately
# and break out the printing into another cell. That way, we are not 
# constantly making API calls to OpenAI etc. Get that done and then 
# experiment with how to print out the data. Otw have some way of 
# memoizing the get_response() function
for h in chat_history:
    role = h["role"]
    txt  = h["content"]

    match role:
        case "user":
            DH.color_box(txt, bg="orange", align=TextAlign.LEFT)
        case _:
            assert(role == "assistant")
            DH.color_box(txt, bg="lightgreen", align=TextAlign.RIGHT)

In [17]:
import json

# json.dumps(chat_history, indent=4) can be used 
# to dump this list. I can just print it out but 
# using Markdown with json syntax highlighting 
# will be nicer
DM.md(DM.json_fmt(chat_history))

```json
[
    {
        "role": "user",
        "content": "Tell me a clean joke about chefs"
    },
    {
        "role": "assistant",
        "content": "Why did the chef break up with their partner?\n\nBecause they just couldn't find the thyme!"
    },
    {
        "role": "user",
        "content": "Summarize it"
    },
    {
        "role": "assistant",
        "content": "A chef broke up with their partner because they couldn't find the time."
    },
    {
        "role": "user",
        "content": "Turn it into a Haiku"
    },
    {
        "role": "assistant",
        "content": "Chef's heart lost in time,  \nLove simmered, but couldn't cook\u2014  \nThyme slipped through their hands."
    }
]
```

# Colab Experiments

The following was taken from google's docs. I then launched this notebook from colab using the https://colab.research.google.com/github/vamsi-juvvi/py-llm/blob/2_lib_use_openai_and_colab/nbs/Py_mod_openai_utils_devel.ipynb link and tested this cell. Once done, transfered this to the top

In [ ]:
# The default relative path
import os
from pathlib import Path
LIB_PATH = Path("../lib")

if 'google.colab' in str(get_ipython()):
    # This will create a py-llm dir at the same level as this notebook
    # Refer to the lib in there using `./py-llm/lib` as opposed to the 
    # relative `../lib` when we are running straight from the py-llm/nbs 
    # directory in VScode.
    if not os.path.isdir("./py-llm"):
        print("Cloning git repo into ./py-llm")
        !git clone -b 2_lib_use_openai_and_colab https://github.com/vamsi-juvvi/py-llm.git
        LIB_PATH = Path("./py-llm/lib")
    else:
        print("./py-llm exists. Not cloning")        

- ✔️ Make sure the above runs with no errors on colab
  - ✔️ Clones correctly
  - ✔️ The `LIB_PATH` is correct and allows us to import the `jpyter_utils` module.
- ✔️ Setup the `OPENAI_API_KEY` key in secrets, enable it for this notebook and ensure that 
  - ✔️ `ColabEnv.colab_keyval("OPENAI_API_KEY")` returns whatever was set
  - ✔️ `ColabEnv.colab_keyval_or_env("OPENAI_API_KEY")` works on colab and in VSCode
- ✔️ Make the changes at the top of the notebook and then run all the cells from Colab.  